In [1]:
import numpy as np
import os
import time
from utils import *

In [2]:
phi = relu
dev_phi = dev_relu

fmap = 'rf'

In [32]:
ks = [10000]
d = 1000
Ns = [3000]
Nt = 100

In [33]:
testr = []
test = []


for N in Ns:
    for k in ks:
    # Dataset
       
        dy = d // 2
        dx = d - dy

        u = np.random.randn(dx) / np.sqrt(dx)

        X = np.random.randn(N, dx)
        Y = np.random.randn(N, dy)
        Z = np.concatenate((X, Y), axis=1)  # training data

        G = np.sign(X @ u)

        Xa = np.random.randn(N, dx)
        Za = np.concatenate((Xa, Y), axis=1)  # attack data

        Xt = np.random.randn(Nt, dx)
        Yt = np.random.randn(Nt, dy)
        Zt = np.concatenate((Xt, Yt), axis=1)  # test data

        Gt = np.sign(Xt @ u)
    
        # Feature Map
    

        V = np.random.randn(k, d) / np.sqrt(d)
        
        Phi = phi(Z @ V.transpose())
        Phi_a = phi(Za @ V.transpose())
        Phi_t = phi(Zt @ V.transpose())
        
        V_inv =  np.linalg.pinv(V.transpose())
        P = V_inv @ V.transpose()
        # Sigma = V_inv @ V_inv.transpose()

    
        # Solution
        # K_r = Phi_1 @ Phi_1.transpose()
        K_inv = np.linalg.inv(Phi @ Phi.transpose())
        pPhi = Phi.transpose() @ K_inv

        # Sigma = np.linalg.inv(d * k / 2 * np.eye(d * k) + Phi_1.transpose() @ Phi_1)
        eps = np.sqrt(d) / (100 * k) * V @ np.random.randn(d)  # np.random.multivariate_normal(np.zeros(k), Sigma)  #  * np.sqrt(d / N)
        
        theta = np.linalg.pinv(Phi) @ G
        theta_1 = 0.5 * V @ Z.transpose() @ K_inv @ G + eps
        theta_2 = P @ theta + eps
    
        # Scores
    
        score_attack = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta) * G[i] > 0:
                score_attack += 1
        score_attack /= N
    
        score_test = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta) * Gt[i] > 0:
                score_test += 1
        score_test /= Nt
    
        score_train = 0  # sanity check
        for i in range(N):
            if np.inner(Phi[i], theta) * G[i] > 0:
                score_train += 1
        score_train /= N
    
    
        # Scores 1
    
        score_attack_1 = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta_1) * G[i] > 0:
                score_attack_1 += 1
        score_attack_1 /= N
    
        score_test_1 = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta_1) * Gt[i] > 0:
                score_test_1 += 1
        score_test_1 /= Nt
    
        score_train_1 = 0
        for i in range(N):
            if np.inner(Phi[i], theta_1) * G[i] > 0:
                score_train_1 += 1
        score_train_1 /= N
    
    
        # Scores 2
    
        score_attack_2 = 0
        for i in range(N):
            if np.inner(Phi_a[i], theta_2) * G[i] > 0:
                score_attack_2 += 1
        score_attack_2 /= N
    
        score_test_2 = 0
        for i in range(Nt):
            if np.inner(Phi_t[i], theta_2) * Gt[i] > 0:
                score_test_2 += 1
        score_test_2 /= Nt
    
        score_train_2 = 0
        for i in range(N):
            if np.inner(Phi[i], theta_2) * G[i] > 0:
                score_train_2 += 1
        score_train_2 /= N
    
    

In [34]:
# z_t = np.random.randn(d)
y = np.random.randn(dy)
z_t = np.concatenate((np.sqrt(dx) * u, y))
phi_z = phi(V @ z_t)

f = theta.transpose() @ phi_z
f

17.474462199834413

In [71]:
phi = relu
dev_phi = dev_relu

gt = 'sign'

k = 10000
d = 100
N = 1000

# xt = np.random.randn(d)
u = np.random.randn(d) / np.sqrt(d)
X = np.random.randn(N, d) #  + np.outer(np.ones(N), xt) / np.sqrt(d)

In [72]:
if 'sign' in gt:
    G = np.sign(X @ u)
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = np.sign(np.inner(xt, u)) # np.sign(Xt @ u)
elif 'linear' in gt:
    G = X @ u
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = np.inner(xt, u) # Xt @ u
elif 'square' in gt:
    G = (X @ u) ** 2
    xt = np.sqrt(d) * X.transpose() @ G / np.linalg.norm(X.transpose() @ G)
    Gt = (np.inner(xt, u)) ** 2 # (Xt @ u) ** 2

V = np.random.randn(k, d) / np.sqrt(d)
Phi = phi(X @ V.transpose())
phi_t = phi(V @ xt)

V_inv =  np.linalg.pinv(V.transpose())
P = V_inv @ V.transpose()

# ort_test = (np.eye(k) - P) @ Phi_t[0]
    
theta = np.linalg.pinv(Phi) @ G
theta_1 = P @ theta


test_loss = np.linalg.norm(np.inner(phi_t, theta) - Gt) ** 2
train_loss = np.linalg.norm(Phi @ theta - G) ** 2 / N
test_loss_1 = np.linalg.norm(np.inner(phi_t, theta_1) - Gt) ** 2
train_loss_1 = np.linalg.norm(Phi @ theta_1 - G) ** 2 / N

In [74]:
test_loss_1

42.20519857915207

array([[-1.60096694,  0.27174977, -0.21172676, ..., -0.48479041,
        -0.96792269,  1.63459134],
       [ 0.13787489,  0.77991542,  0.25285986, ..., -0.94782649,
         0.8341446 , -0.35706429],
       [-0.47516119,  0.61757114, -0.13649291, ...,  0.07088566,
         1.17678482,  1.75481149],
       ...,
       [-0.59335394, -1.00680831,  0.42989737, ...,  2.33793473,
         0.29115917,  0.47490082],
       [-0.12345399,  0.32266323, -0.12814617, ...,  2.43098824,
         1.74532975,  0.2274105 ],
       [ 1.00606762,  0.67662096,  0.8584355 , ...,  0.44042554,
         1.9993179 ,  0.14236148]])

In [3]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
import argparse
import time
import os
import json
import sys

In [4]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--k')
# args = parser.parse_args()

# k = int(args.k)
# time.sleep(k)

k = 0

In [5]:
save_dir = os.path.join('NN', '01_09_fullbatch_3')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=60000, shuffle=True, num_workers=1, pin_memory=True  # Full batch
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10000, shuffle=True, num_workers=1, pin_memory=True
)

In [6]:
class SimpleFCN(nn.Module):
    def __init__(self, width):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, width, bias=False)  # no bias
        self.fc2 = nn.Linear(width, 10, bias=False)  # no bias
        # nn.init.zeros_(self.fc2.weight)  # 0 init

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def train_full_batch(model, train_loader, optimizer, criterion, device, sample_size=100):
    model.train()
    param1_grad_norms = []
    param2_grad_norms = []

    # Get the full batch of data
    data, target = next(iter(train_loader))
    data, target = data.to(device), target.to(device)

    # Calculate the gradient norms for a subset of the samples
    for i in range(sample_size):
        sample_grad_norm_param1 = 0
        sample_grad_norm_param2 = 0
        model.zero_grad()  # Reset gradients for each sample
        output = model(data[i].unsqueeze(0))  # Forward pass for the single sample
        loss = criterion(output, target[i].unsqueeze(0))  # Compute loss for the sample
        loss.backward()  # Compute gradients for this sample
    
        for j, param in enumerate(model.parameters()):
            if param.grad is not None:
                if j == 0:  # First parameter (hidden layer)
                    sample_grad_norm_param1 += param.grad.norm().item() ** 2
                elif j == 1:  # Second parameter (output layer)
                    sample_grad_norm_param2 += param.grad.norm().item() ** 2

        param1_grad_norms.append(np.sqrt(sample_grad_norm_param1))
        param2_grad_norms.append(np.sqrt(sample_grad_norm_param2))

    # After computing the per-sample gradients, perform the actual full-batch optimization step
    optimizer.zero_grad()  # Reset gradients before full batch forward pass
    output = model(data)
    loss = criterion(output, target)
    loss.backward()  # Compute gradients for the full batch

    optimizer.step()  # Perform the optimizer step after computing gradients for the full batch

    avg_grad_norm_param1 = np.mean(param1_grad_norms)
    avg_grad_norm_param2 = np.mean(param2_grad_norms)

    return loss.item(), avg_grad_norm_param1, avg_grad_norm_param2


def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, accuracy

In [9]:
# widths = [100, 200, 500, 1000, 2000, 5000, 10000]
# width = widths[k % 7]
# k = k // 7

width = 1000

model = SimpleFCN(width)

# Clrs = [0.001, 0.01, 0.1, 1, 10]
# Clr = Clrs[k % 5]
# k = k // 5
# lr = 1000 * Clr / width

lr = 0.1

optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

train_losses = []
test_losses = []
test_accuracies = []
grad_norms_param1 = []
grad_norms_param2 = []

T = 100

cuda


In [10]:
for epoch in range(1, T + 1):
    print(f"Epoch {epoch}")
    train_loss, avg_grad_norm_param1, avg_grad_norm_param2 = train_full_batch(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = test(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    grad_norms_param1.append(avg_grad_norm_param1)
    grad_norms_param2.append(avg_grad_norm_param2)
    
    print(f"Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}, Accuracy: {test_accuracy:.2f}%")
    print(f"Avg Grad Norm Param1: {avg_grad_norm_param1:.6f}, Avg Grad Norm Param2: {avg_grad_norm_param2:.6f}")

Epoch 1
Train Loss: 2.284513, Test Loss: 1.985296, Accuracy: 55.13%
Avg Grad Norm Param1: 10.885303, Avg Grad Norm Param2: 12.444546
Epoch 2
Train Loss: 1.990868, Test Loss: 1.744897, Accuracy: 68.12%
Avg Grad Norm Param1: 10.183039, Avg Grad Norm Param2: 11.547263
Epoch 3
Train Loss: 1.755890, Test Loss: 1.538762, Accuracy: 73.27%
Avg Grad Norm Param1: 10.114790, Avg Grad Norm Param2: 11.408100
Epoch 4
Train Loss: 1.554209, Test Loss: 1.363927, Accuracy: 76.77%
Avg Grad Norm Param1: 9.439765, Avg Grad Norm Param2: 10.746394
Epoch 5
Train Loss: 1.382727, Test Loss: 1.218139, Accuracy: 79.04%
Avg Grad Norm Param1: 9.116814, Avg Grad Norm Param2: 10.384436
Epoch 6
Train Loss: 1.239348, Test Loss: 1.098219, Accuracy: 80.57%
Avg Grad Norm Param1: 8.621726, Avg Grad Norm Param2: 9.720040
Epoch 7
Train Loss: 1.121013, Test Loss: 1.000088, Accuracy: 81.63%
Avg Grad Norm Param1: 8.707281, Avg Grad Norm Param2: 9.888912
Epoch 8
Train Loss: 1.023869, Test Loss: 0.919686, Accuracy: 82.57%
Avg Gra

KeyboardInterrupt: 

In [18]:
model.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)

    print(target[:5])
    
    output = model(data)

    print(output[:5])
    
    test_loss += criterion(output, target).item()

    print(criterion(output, target).item())
    
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

tensor([7, 2, 1, 0, 4])
tensor([[-0.0074,  0.0664, -0.0074, -0.0074, -0.0074, -0.0074, -0.0074, -0.0074,
         -0.0074, -0.0074],
        [-0.0072,  0.0652, -0.0072, -0.0072, -0.0072, -0.0072, -0.0072, -0.0072,
         -0.0072, -0.0072],
        [-0.0100,  0.0899, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100,
         -0.0100, -0.0100],
        [-0.0085,  0.0769, -0.0085, -0.0085, -0.0085, -0.0085, -0.0085, -0.0085,
         -0.0085, -0.0085],
        [-0.0043,  0.0385, -0.0043, -0.0043, -0.0043, -0.0043, -0.0043, -0.0043,
         -0.0043, -0.0043]])
2.2997887134552


In [19]:
len(test_loader.dataset)

10000

In [ ]:
# Save results to JSON file
results = {
    'width': width,
    'Clr': Clr,
    'train_losses': train_losses,
    'test_losses': test_losses,
    'test_accuracies': test_accuracies
    'grad_norms_param1': grad_norms_param1,
    'grad_norms_param2': grad_norms_param2
}

with open(os.path.join(save_dir, f'results_{args.k}.json'), 'w') as f:
    json.dump(results, f)

SyntaxError: invalid syntax (162238052.py, line 155)

In [ ]:
def compute_clipped_gradients(sample_data, sample_target, model, criterion, clip_values, device):
    
    sample_data, sample_target = sample_data.to(device), sample_target.to(device)  # no batch dimension
    
    output = model(sample_data.unsqueeze(0))
    loss = criterion(output, sample_target.unsqueeze(0))
    
    total_grads = [torch.zeros_like(param) for param in model.parameters()]

    model.zero_grad()
    grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True)  # why retain graph?

    for j, grad in enumerate(grads):
        grad_norm = grad.norm()
        clip_value = clip_values[j]
        if grad_norm > clip_value:
            grad = grad * (clip_value / grad_norm)
        grads[j] = grad
    
    return grads


def train_dp_batch(model, train_loader, lr, n, criterion, device, clip_values, sigma):

    model.train()

    for data, target in train_loader:  # batch-size > 1
        
        batch_grads = compute_clipped_gradients(data, target, model, criterion, clip_values, device)
        # seems wrong, more dimensions only in first 2 args

        mean_grad = # compute mean over batch dimension

        for i, param in enumerate(model.parameters()):
            noise = np.sqrt(lr) * (2 * clip_values[i] / n) * sigma * torch.normal(0, 1, size=param.grad.size()).to(device)
            param.data = param.data - lr * mean_grad[i] + noise

In [1]:
!pip install jax

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import jax
import jax.numpy as jnp
import optax
from jax import grad, jit, vmap
import numpy as np
import os
import argparse
import time
import json
from functools import partial
from tensorflow.keras.datasets import mnist

# # Argument parsing
# parser = argparse.ArgumentParser()
# parser.add_argument('--k')
# args = parser.parse_args()

# k = int(args.k)
# time.sleep(k)


k = 0

save_dir = os.path.join('NN', '04_09_grid_jax')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(-1, 28 * 28) / 255.0
test_images = test_images.reshape(-1, 28 * 28) / 255.0

# Create batches
def create_batches(images, labels, batch_size):
    n_batches = len(images) // batch_size
    for i in range(n_batches):
        yield images[i * batch_size:(i + 1) * batch_size], labels[i * batch_size:(i + 1) * batch_size]

# Initialize parameters for the simple fully connected network
def init_params(width, key):
    key1, key2 = jax.random.split(key)
    w1 = jax.random.normal(key1, (28 * 28, width))
    w2 = jax.random.normal(key2, (width, 10))
    return w1, w2

# Forward pass
def forward(params, x):
    w1, w2 = params
    h = jax.nn.relu(jnp.dot(x, w1))
    return jnp.dot(h, w2)

# Loss function (cross-entropy)
def loss_fn(params, x, y):
    logits = forward(params, x)
    one_hot = jax.nn.one_hot(y, 10)
    return jnp.mean(optax.softmax_cross_entropy(logits, one_hot))

# Gradient clipping
def clip_grads(grads, clip_values):
    clipped_grads = []
    for grad, clip_value in zip(grads, clip_values):
        norm = jnp.linalg.norm(grad)
        factor = jnp.minimum(1.0, clip_value / (norm + 1e-6))
        clipped_grads.append(grad * factor)
    return clipped_grads

# Training step
@jit
def train_step(params, x, y, lr, clip_values, sigma, key):
    grads = grad(loss_fn)(params, x, y)
    grads = clip_grads(grads, clip_values)
    noise = [(jax.random.normal(key, g.shape) * sigma) for g in grads]
    updated_params = [(w - lr * (g + n)) for w, g, n in zip(params, grads, noise)]
    return updated_params

# Test function
@jit
def test_fn(params, x, y):
    predictions = forward(params, x)
    predicted_labels = jnp.argmax(predictions, axis=1)
    accuracy = jnp.mean(predicted_labels == y)
    return accuracy

# # Main training loop
# widths = [100, 1000, 10000, 100000]
# width = widths[k % 4]
# k = k // 4

width = 1000

lr = 0.1
n = 60000
delta = 1 / n
# epsilons = [1, 4]
# eps = epsilons[k % 2]
# k = k // 2
eps = 4


# clip_values_1 = np.logspace(0, 1, 5)
# clip_value_1 = clip_values_1[k % 5]
# k = k // 5

clip_value_1 = 1

# C2s = np.logspace(-1, 0, 5)
# C2 = C2s[k % 5]
# k = k // 5

C2 = 0.5

clip_value_2 = C2 * np.sqrt(width)
clip_values = [clip_value_1, clip_value_2]

# Tps = np.logspace(1, 3, 5)

# for Tp in Tps:


# T = int(Tp * np.sqrt(10000 / width))

T = 100

print("------------------------------------")
print(f"Training with T={T}")
print("------------------------------------")

key = jax.random.PRNGKey(0)
params = init_params(width, key)

sigma = np.sqrt(2) * np.sqrt(lr * T) * np.sqrt(8 * jnp.log(1 / delta)) / eps

for epoch in range(1, T + 1):
    for batch_x, batch_y in create_batches(train_images, train_labels, batch_size=128):
        key, subkey = jax.random.split(key)
        params = train_step(params, batch_x, batch_y, lr, clip_values, sigma, subkey)

    # Test accuracy every 10 epochs
    if epoch % 10 == 0:
        accuracy = test_fn(params, test_images, test_labels)
        print(f"Epoch {epoch}, Test Accuracy: {accuracy * 100:.2f}%")

with open(os.path.join(save_dir, 'parameters_' + args.k + '.txt'), 'a') as f:
    f.write(f"{n}\t{width}\t{accuracy:.4f}\t{eps}\t{T}\t{clip_value_1:.2f}\t{clip_value_2:.2f}\n")


------------------------------------
Training with T=100
------------------------------------
Epoch 10, Test Accuracy: 69.80%


KeyboardInterrupt: 

In [8]:
import jax
print(jax.devices())

[CpuDevice(id=0)]


In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap, nn
import optax
import tensorflow_datasets as tfds
import jax.random as jrandom
import jax
from jax.tree_util import tree_map, tree_leaves


def load_mnist():
    try:
        (train_data, train_labels), (test_data, test_labels) = tfds.load('mnist', split=['train', 'test'], batch_size=-1, as_supervised=True)
        train_data = jnp.array(train_data, dtype=jnp.float32) / 255.0
        test_data = jnp.array(test_data, dtype=jnp.float32) / 255.0
        
        # Flatten images
        train_data = train_data.reshape(train_data.shape[0], -1)
        test_data = test_data.reshape(test_data.shape[0], -1)
        
        # Normalize
        mean = jnp.mean(train_data, axis=0)
        std = jnp.std(train_data, axis=0)
        train_data = (train_data - mean) / (std + 1e-8)
        test_data = (test_data - mean) / (std + 1e-8)
        
        train_labels = jnp.array(train_labels)
        test_labels = jnp.array(test_labels)
        
        return train_data, train_labels, test_data, test_labels
    except Exception as e:
        print(f"Error loading MNIST dataset: {e}")
        return None, None, None, None


def initialize_params(input_dim, hidden_dim, output_dim, key):
    key1, key2 = jrandom.split(key)
    
    # He initialization
    stddev1 = jnp.sqrt(2.0 / input_dim)
    stddev2 = jnp.sqrt(2.0 / hidden_dim)

    V_1 = jrandom.normal(key1, (hidden_dim, input_dim)) * stddev1
    V_2 = jrandom.normal(key2, (output_dim, hidden_dim)) * stddev2

    return (V_1, V_2)


def predict(params, x):
    V_1, V_2 = params
    pre_act = jnp.matmul(x, V_1.T)
    act = nn.relu(pre_act)
    out = jnp.matmul(act, V_2.T)
    return out


def loss(params, x, target):
    pred = predict(params, x)
    return optax.softmax_cross_entropy_with_integer_labels(pred, target)


def clip_gradient(g, clip_norm):
    grad_norm = jnp.linalg.norm(g)
    return jnp.where(grad_norm <= clip_norm, g, g * (clip_norm / grad_norm))


def evaluate(params, data, labels):
    predictions = predict(params, data)
    predicted_labels = jnp.argmax(predictions, axis=1)
    accuracy = jnp.mean(predicted_labels == labels)
    return accuracy


@jit
def accumulate_grads(accumulated_grads, batch_grads):
    return tree_map(lambda acc, batch: acc + batch, accumulated_grads, batch_grads)


def compute_batch_clipped_grads(params, x, y, clip_norms, batch_idx):
    def single_sample_loss(params, x, y):
        pred = predict(params, x)
        return optax.softmax_cross_entropy_with_integer_labels(pred, y)

    batch_grad_fn = vmap(grad(single_sample_loss), in_axes=(None, 0, 0))
    per_sample_grads = batch_grad_fn(params, x, y)

    # Clip gradients layer-wise
    per_sample_clipped_grads = tree_map(lambda g, c: vmap(clip_gradient, in_axes=(0, None))(g, c),
                                        per_sample_grads, clip_norms)


    # Debugging: Print information for the first few samples
    # def debug_gradients(grads, clipped_grads):
    #     for i in range(min(10, x.shape[0])):
    #         print(f"Sample {i}:")
    #         for j, (g, cg) in enumerate(zip(tree_leaves(grads), tree_leaves(clipped_grads))):
    #             print(f"  Param {j}:")
    #             print(f"    Original norm: {jnp.linalg.norm(g[i]):.4f}")
    #             print(f"    Clipped norm: {jnp.linalg.norm(cg[i]):.4f}")
    #             print(f"    Clip norm: {clip_norms[j]:.4f}")
    #         print()

    # print("Debugging gradient information:")
    # debug_gradients(per_sample_grads, per_sample_clipped_grads)
    
    # Sum clipped gradients for this batch
    batch_sum_clipped_grads = tree_map(lambda g: jnp.sum(g, axis=0), per_sample_clipped_grads)

    if batch_idx == 0:
        avg_grad_norms = tree_map(lambda g: jnp.mean(jnp.linalg.norm(g, axis=(-2, -1))), per_sample_clipped_grads)
        return batch_sum_clipped_grads, avg_grad_norms
    else:
        return batch_sum_clipped_grads, None


def training_step_full(params, train_data, train_labels, learning_rate, noise_multiplier, clip_norms, key, batch_size):
    num_samples = train_data.shape[0]
    num_batches = num_samples // batch_size
    accumulated_grads = tree_map(lambda p: jnp.zeros_like(p), params)
    first_batch_avg_grad_norms = None
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = train_data[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]
        batch_grads, avg_grad_norms = compute_batch_clipped_grads(params, batch_data, batch_labels, clip_norms, i)
        accumulated_grads = accumulate_grads(accumulated_grads, batch_grads)
        
        if i == 0:
            first_batch_avg_grad_norms = avg_grad_norms
    
    avg_grads = tree_map(lambda g: g / num_samples, accumulated_grads)
    keys = tuple(jrandom.split(key, len(tree_leaves(avg_grads))))
    noises = tree_map(
        lambda c, k, g: noise_multiplier * c * jrandom.normal(k, g.shape),
        clip_norms, keys, avg_grads
    )
    new_params = tree_map(lambda p, g, n: p - learning_rate * g + n, params, avg_grads, noises)
    return new_params, first_batch_avg_grad_norms

        
def training_step_no_noise(params, train_data, train_labels, learning_rate, clip_norms, batch_size):
    num_samples = train_data.shape[0]
    num_batches = num_samples // batch_size
    accumulated_grads = tree_map(lambda p: jnp.zeros_like(p), params)
    first_batch_avg_grad_norms = None
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = train_data[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]
        batch_grads, avg_grad_norms = compute_batch_clipped_grads(params, batch_data, batch_labels, clip_norms, i)
        accumulated_grads = accumulate_grads(accumulated_grads, batch_grads)
        
        if i == 0:
            first_batch_avg_grad_norms = avg_grad_norms
    
    avg_grads = tree_map(lambda g: g / num_samples, accumulated_grads)
    new_params = tree_map(lambda p, g: p - learning_rate * g, params, avg_grads)
    return new_params, first_batch_avg_grad_norms



def training_step_no_clip(params, train_data, train_labels, learning_rate, batch_size):
    def compute_batch_grads(params, x, y, batch_idx):
        def single_sample_loss(params, x, y):
            pred = predict(params, x)
            return optax.softmax_cross_entropy_with_integer_labels(pred, y)
        batch_grad_fn = vmap(grad(single_sample_loss), in_axes=(None, 0, 0))
        batch_grads = batch_grad_fn(params, x, y)
        
        if batch_idx == 0:
            avg_grad_norms = tree_map(lambda g: jnp.mean(jnp.linalg.norm(g, axis=(-2, -1))), batch_grads)
            return jnp.sum(batch_grads, axis=0), avg_grad_norms
        else:
            return jnp.sum(batch_grads, axis=0), None
    
    num_samples = train_data.shape[0]
    num_batches = num_samples // batch_size
    accumulated_grads = tree_map(lambda p: jnp.zeros_like(p), params)
    first_batch_avg_grad_norms = None
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = train_data[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]
        batch_grads, avg_grad_norms = compute_batch_grads(params, batch_data, batch_labels, i)
        accumulated_grads = accumulate_grads(accumulated_grads, batch_grads)
        
        if i == 0:
            first_batch_avg_grad_norms = avg_grad_norms
    
    avg_grads = tree_map(lambda g: g / num_samples, accumulated_grads)
    new_params = tree_map(lambda p, g: p - learning_rate * g, params, avg_grads)
    return new_params, first_batch_avg_grad_norms



def training_step_no_clip_with_noise(params, train_data, train_labels, learning_rate, noise_multiplier, key, batch_size):
    def compute_batch_grads(params, x, y, batch_idx):
        def single_sample_loss(params, x, y):
            pred = predict(params, x)
            return optax.softmax_cross_entropy_with_integer_labels(pred, y)
        batch_grad_fn = vmap(grad(single_sample_loss), in_axes=(None, 0, 0))
        batch_grads = batch_grad_fn(params, x, y)
        
        if batch_idx == 0:
            avg_grad_norms = tree_map(lambda g: jnp.mean(jnp.linalg.norm(g, axis=(-2, -1))), batch_grads)
            return jnp.sum(batch_grads, axis=0), avg_grad_norms
        else:
            return jnp.sum(batch_grads, axis=0), None
    
    num_samples = train_data.shape[0]
    num_batches = num_samples // batch_size
    accumulated_grads = tree_map(lambda p: jnp.zeros_like(p), params)
    first_batch_avg_grad_norms = None
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_data = train_data[start_idx:end_idx]
        batch_labels = train_labels[start_idx:end_idx]
        batch_grads, avg_grad_norms = compute_batch_grads(params, batch_data, batch_labels, i)
        accumulated_grads = accumulate_grads(accumulated_grads, batch_grads)
        
        if i == 0:
            first_batch_avg_grad_norms = avg_grad_norms
    
    avg_grads = tree_map(lambda g: g / num_samples, accumulated_grads)
    keys = tuple(jrandom.split(key, len(tree_leaves(avg_grads))))
    noises = tree_map(
        lambda k, g: noise_multiplier * jrandom.normal(k, g.shape),
        keys, avg_grads
    )
    new_params = tree_map(lambda p, g, n: p - learning_rate * g + n, params, avg_grads, noises)
    return new_params, first_batch_avg_grad_norms




def train(params, learning_rate, train_data, train_labels, test_data, test_labels, num_epochs, batch_size, noise_multiplier, clip_norms, print_every, training_mode='full'):
    key = jrandom.PRNGKey(0)
    metrics = {'train_loss': [], 'test_acc': [], 'grad_norms': []}

    for epoch in range(num_epochs):
        key, subkey = jrandom.split(key)
        
        if training_mode == 'full':
            params, avg_grad_norms = training_step_full(params, train_data, train_labels, learning_rate, noise_multiplier, clip_norms, subkey, batch_size)
        elif training_mode == 'no_noise':
            params, avg_grad_norms = training_step_no_noise(params, train_data, train_labels, learning_rate, clip_norms, batch_size)
        elif training_mode == 'no_clip':
            params, avg_grad_norms = training_step_no_clip(params, train_data, train_labels, learning_rate, batch_size)
        elif training_mode == 'no_clip_with_noise':
            params, avg_grad_norms = training_step_no_clip_with_noise(params, train_data, train_labels, learning_rate, noise_multiplier, subkey, batch_size)
        else:
            raise ValueError("Invalid training mode")

        train_loss = jnp.mean(loss(params, train_data, train_labels))
        test_acc = evaluate(params, test_data, test_labels)
        
        metrics['train_loss'].append(train_loss)
        metrics['test_acc'].append(test_acc)
        metrics['grad_norms'].append(avg_grad_norms)

        if epoch % print_every == 0:
            print(f"Epoch {epoch}: Train loss = {train_loss:.4f}, Test accuracy = {test_acc:.4f}")
            print(f"Average gradient norms: {avg_grad_norms}")

    return params, metrics



train_data, train_labels, test_data, test_labels = load_mnist()

# Initialize parameters
input_dim = train_data.shape[1]
hidden_dim = 1000
output_dim = 10
key = jrandom.PRNGKey(0)
# print(key)
params = initialize_params(input_dim, hidden_dim, output_dim, key)

# Training hyperparameters
lr = 1
batch_size = 100
num_samples = 60000
T = 5
clip_norm_v1 = 2
clip_norm_v2 = 5
print_every = 1
num_epochs = T

eps = 4
delta = 1 / num_samples

sigma = jnp.sqrt(2) * jnp.sqrt(lr * T) * jnp.sqrt(8 * jnp.log(1 / delta)) / eps
noise_multiplier = jnp.sqrt(lr) * (2 * 1 / num_samples) * sigma  # The 1 is the clipping constant inside the function 


# Train the model
final_params, metrics = train(params, lr, train_data, train_labels, test_data, test_labels,
              num_epochs, batch_size, noise_multiplier, (clip_norm_v1, clip_norm_v2), print_every, training_mode='full')

# Evaluate final model
final_accuracy = evaluate(final_params, test_data, test_labels)
print(f"Final test accuracy: {final_accuracy:.4f}")

# You can now access the metrics dictionary for plotting or further analysis
print("Training losses:", metrics['train_loss'])
print("Test accuracies:", metrics['test_acc'])
print("Average gradient norms:", metrics['grad_norms'])

2024-09-09 09:04:57.447987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 09:04:58.467117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 09:04:58.702488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 09:05:38.206569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 0: Train loss = 2.1503, Test accuracy = 0.3482
Average gradient norms: (Array(2., dtype=float32), Array(5., dtype=float32))
Epoch 1: Train loss = 1.4947, Test accuracy = 0.5602
Average gradient norms: (Array(1.8493034, dtype=float32), Array(4.4057155, dtype=float32))
Epoch 2: Train loss = 1.2346, Test accuracy = 0.6339
Average gradient norms: (Array(1.9148229, dtype=float32), Array(4.600794, dtype=float32))
Epoch 3: Train loss = 1.0162, Test accuracy = 0.7323
Average gradient norms: (Array(1.516403, dtype=float32), Array(3.4498107, dtype=float32))
Epoch 4: Train loss = 0.9112, Test accuracy = 0.7585
Average gradient norms: (Array(1.6075107, dtype=float32), Array(3.6223037, dtype=float32))
Final test accuracy: 0.7585
Training losses: [Array(2.1503034, dtype=float32), Array(1.4947168, dtype=float32), Array(1.234557, dtype=float32), Array(1.0161875, dtype=float32), Array(0.9112235, dtype=float32)]
Test accuracies: [Array(0.3482, dtype=float32), Array(0.5602, dtype=float32), Array(0.

In [1]:
def load_mnist():
    try:
        from sklearn.datasets import fetch_openml
        from sklearn.model_selection import train_test_split
        
        # Load MNIST from OpenML
        X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
        
        # Convert to float32 and scale to [0, 1]
        X = X.astype(np.float32) / 255.0
        
        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42)
        
        # Convert to JAX arrays
        train_data = jnp.array(X_train)
        test_data = jnp.array(X_test)
        train_labels = jnp.array(y_train.astype(int))
        test_labels = jnp.array(y_test.astype(int))
        
        # Normalize
        mean = jnp.mean(train_data, axis=0)
        std = jnp.std(train_data, axis=0)
        train_data = (train_data - mean) / (std + 1e-8)
        test_data = (test_data - mean) / (std + 1e-8)
        
        return train_data, train_labels, test_data, test_labels
    
    except ImportError:
        print("Scikit-learn not found!")
        return None, None, None, None

In [2]:
a, _, _, _ = load_mnist()

Scikit-learn not found!


In [4]:
import os
import numpy as np
import jax.numpy as jnp
from PIL import Image
from torchvision.datasets import MNIST

def load_mnist_local(data_dir='./data'):
    # Load the MNIST dataset from local files
    mnist_train = MNIST(root=data_dir, train=True, download=False)
    mnist_test = MNIST(root=data_dir, train=False, download=False)

    # Convert images and labels to NumPy arrays
    X_train = np.array([np.array(image) for image in mnist_train.data], dtype=np.float32)
    y_train = np.array(mnist_train.targets, dtype=int)
    X_test = np.array([np.array(image) for image in mnist_test.data], dtype=np.float32)
    y_test = np.array(mnist_test.targets, dtype=int)

    # Normalize data to [0, 1]
    X_train /= 255.0
    X_test /= 255.0

    # Reshape data from (60000, 28, 28) to (60000, 784) to match sklearn's format
    X_train = X_train.reshape((X_train.shape[0], -1))
    X_test = X_test.reshape((X_test.shape[0], -1))

    # Convert to JAX arrays
    train_data = jnp.array(X_train)
    test_data = jnp.array(X_test)
    train_labels = jnp.array(y_train)
    test_labels = jnp.array(y_test)

    # Normalize using JAX
    mean = jnp.mean(train_data, axis=0)
    std = jnp.std(train_data, axis=0)
    train_data = (train_data - mean) / (std + 1e-8)
    test_data = (test_data - mean) / (std + 1e-8)

    return train_data, train_labels, test_data, test_labels

# Example usage
train_data, train_labels, test_data, test_labels = load_mnist_local()
print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)


2024-09-09 10:50:08.425500: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.68). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


(60000, 784) (60000,) (10000, 784) (10000,)
